In [1]:
import os
import requests
from bs4 import BeautifulSoup
from docx import Document
import zipfile
import re
import aspose.words as aw


In [2]:
import pdfplumber

TÁCH VĂN BẢN THEO CÁC ĐIỀU LUẬT

In [3]:
import os
import pythoncom
from win32com.client import Dispatch

def split_doc_by_rules(input_file, output_folder):
    # Tạo thư mục đích nếu chưa tồn tại
    os.makedirs(output_folder, exist_ok=True)
    
    # Khởi tạo COM object để tương tác với Microsoft Word
    pythoncom.CoInitialize()
    word = Dispatch("Word.Application")
    doc = word.Documents.Open(input_file)
    
    # Đọc toàn bộ nội dung của file
    full_text = [para.Range.Text.strip() for para in doc.Paragraphs if para.Range.Text.strip() != '']
    
    # Đóng file và Word application
    doc.Close()
    word.Quit()
    
    # Tách văn bản thành các đoạn theo từ khóa "Điều"
    dieu_sections = []
    current_section = []
    
    for paragraph in full_text:
        if paragraph.startswith("Điều"):
            if current_section:
                dieu_sections.append(current_section)
                current_section = []
        current_section.append(paragraph)
    if current_section:
        dieu_sections.append(current_section)
    
    # Ghi từng phần vào các file riêng biệt
    for i, section in enumerate(dieu_sections):
        output_file = os.path.join(output_folder, f"Dieu_{i+1}.txt")
        
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write("\n".join(section))
    
    print(f"Đã tách file {input_file} thành {len(dieu_sections)} phần trong thư mục {output_folder}.")

tách pdf theo điều khoản


In [4]:


def split_pdf_by_rules(input_file, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    
    full_text = []
    with pdfplumber.open(input_file) as pdf:
        for page in pdf.pages:
            full_text.extend(page.extract_text().split('\n'))
    
    content = "\n".join(full_text)
    
    sections = re.split(r'(Điều \d+)', content)
    
    combined_sections = []
    for i in range(1, len(sections), 2):
        combined_sections.append(sections[i] + sections[i+1])
    
    for i, section in enumerate(combined_sections):
        match = re.search(r'Điều (\d+)', section)
        dieu_number = match.group(1) if match else str(i + 1)
        output_file = os.path.join(output_folder, f"Dieu_{dieu_number}.txt")
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(section)
    
    print(f"Đã tách file {input_file} thành {len(combined_sections)} phần trong thư mục {output_folder}.")

TÁCH TXT THEO ĐIỀU KHOẢN

In [25]:
def split_txt_by_rules(input_file, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    
    with open(input_file, 'r', encoding='utf-8') as file:
        content = file.read()
    
    sections = re.split(r'(Điều \d+)', content)
    
    combined_sections = []
    for i in range(1, len(sections), 2):
        combined_sections.append(sections[i] + sections[i + 1])
    
    for i, section in enumerate(combined_sections):
        match = re.search(r'Điều (\d+)', section)
        dieu_number = match.group(1) if match else str(i + 1)
        output_file = os.path.join(output_folder, f"Dieu_{dieu_number}.txt")
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(section)
    
    print(f"Đã tách file {input_file} thành {len(combined_sections)} phần trong thư mục {output_folder}.")

RENAME FILE

In [7]:
def rename_file(folder_path):

    for file in os.listdir(folder_path):
        if file.endswith('.txt'):
            file_name = 'new_' + file_name
            
            old_file = os.path.join(folder_path,file)
            new_file = os.path.join(folder_path,file_name)

            os.rename(old_file,new_file)






DOCX TO TEXT

In [5]:
def docx_to_txt(input_file, output_file):
    doc = Document(input_file)
        
    full_text = [para.text for para in doc.paragraphs if para.text.strip() != '']
    
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write("\n".join(full_text))
    
    print(f"Đã chuyển đổi file {input_file} thành {output_file}.")

PDF TO TEXT

In [5]:
def pdf_to_txt(pdf_dir,output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for file_name in os.listdir(pdf_dir):
        if file_name.lower().endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, file_name)
            txt_file_name = f'{os.path.splitext(file_name)[0]}.txt'
            txt_path = os.path.join(output_dir, txt_file_name)

            with open(txt_path, 'w', encoding='utf-8') as txt_file:
                with pdfplumber.open(pdf_path) as pdf:
                    for page_num, page in enumerate(pdf.pages):
                        content = page.extract_text()
                        if content:  # Kiểm tra nếu có nội dung
                            txt_file.write(f'Nội dung của {file_name} - Trang {page_num + 1}:\n{content}\n\n')


CHUYEN DOCX DOC SANG TÊXT VA TACH THANH NHIEU FILE TXT

In [6]:
import os
from docx import Document

def split_docx_by_lines(input_file, output_folder, lines_per_file):
    os.makedirs(output_folder, exist_ok=True)
    
    doc = Document(input_file)
    
    full_text = [para.text for para in doc.paragraphs if para.text.strip() != '']
    
    total_lines = len(full_text)
    total_files = (total_lines // lines_per_file) + (1 if total_lines % lines_per_file != 0 else 0)

    for i in range(total_files):
        start_line = i * lines_per_file
        end_line = min((i + 1) * lines_per_file, total_lines)
        file_lines = full_text[start_line:end_line]
        
        output_file = os.path.join(output_folder, f"Trang_{i+1}.txt")
        
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write("\n".join(file_lines))
    
    print(f"Đã tách file {input_file} thành {total_files} phần trong thư mục {output_folder}.")

TÁCH DOC NHIÈU FILE TXT THEO DÒNG

In [7]:
import os
import pythoncom
from win32com.client import Dispatch

def split_doc_by_lines(input_file, output_folder, lines_per_file):
    os.makedirs(output_folder, exist_ok=True)
    
    pythoncom.CoInitialize()
    word = Dispatch("Word.Application")
    doc = word.Documents.Open(input_file)
    
    full_text = []
    for para in doc.Paragraphs:
        text = para.Range.Text.strip()
        if text:
            full_text.append(text)
    
    doc.Close()
    word.Quit()
    
    total_lines = len(full_text)
    total_files = (total_lines // lines_per_file) + (1 if total_lines % lines_per_file != 0 else 0)

    for i in range(total_files):
        start_line = i * lines_per_file
        end_line = min((i + 1) * lines_per_file, total_lines)
        file_lines = full_text[start_line:end_line]
        
        output_file = os.path.join(output_folder, f"Trang_{i+1}.txt")
        
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write("\n".join(file_lines))
    
    print(f"Đã tách file {input_file} thành {total_files} phần trong thư mục {output_folder}.")

TÁCH TXT THEO DÒNG

In [8]:
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
    result = chardet.detect(raw_data)
    return result['encoding']

def split_txt_by_lines(input_file, output_folder, lines_per_file):
    encoding = detect_encoding(input_file)

    os.makedirs(output_folder, exist_ok=True)
    
    with open(input_file, 'r', encoding=encoding) as file:
        lines = file.readlines()
    
    total_lines = len(lines)
    total_files = (total_lines // lines_per_file) + (1 if total_lines % lines_per_file != 0 else 0)

    # Tạo các file theo số lượng dòng
    for i in range(total_files):
        start_line = i * lines_per_file
        end_line = min((i + 1) * lines_per_file, total_lines)
        file_lines = lines[start_line:end_line]
        
        # Tạo tên file cho từng phần
        output_file = os.path.join(output_folder, f"Trang_{i+1}.txt")
        
        # Ghi nội dung vào file
        with open(output_file, 'w', encoding='utf-8') as file:
            file.writelines(file_lines)
    
    print(f"Đã tách file {input_file} thành {total_files} phần trong thư mục {output_folder}.")

TÁCH TXT THEO TRANG

In [9]:
def split_txt_by_pages(input_file, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    
    with open(input_file, 'r', encoding='utf-8') as file:
        content = file.read()
    
    pages = re.split(r'(Trang \d+)', content)
    
    combined_pages = []
    for i in range(1, len(pages), 2):
        combined_pages.append(pages[i] + pages[i+1])
    
    for page in combined_pages:
        match = re.search(r'Trang (\d+)', page)
        page_number = match.group(1) if match else str(i+1)
        output_file = os.path.join(output_folder, f"Trang_{page_number}.txt")
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(page)
    
    print(f"Đã tách file {input_file} thành {len(combined_pages)} trang trong thư mục {output_folder}.")


TÁCH DOC THEO TRANG

In [10]:
import os
import pythoncom
from win32com.client import Dispatch

def split_doc_by_lines(input_file, output_folder, lines_per_file):
    os.makedirs(output_folder, exist_ok=True)
    
    pythoncom.CoInitialize()
    word = Dispatch("Word.Application")
    doc = word.Documents.Open(input_file)
    
    full_text = [para.Range.Text.strip() for para in doc.Paragraphs if para.Range.Text.strip() != '']
    
    doc.Close()
    word.Quit()
    
    total_lines = len(full_text)
    total_files = (total_lines // lines_per_file) + (1 if total_lines % lines_per_file != 0 else 0)

    for i in range(total_files):
        start_line = i * lines_per_file
        end_line = min((i + 1) * lines_per_file, total_lines)
        file_lines = full_text[start_line:end_line]
        
        output_file = os.path.join(output_folder, f"Trang_{i+1}.txt")
        
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write("\n".join(file_lines))
    
    print(f"Đã tách file {input_file} thành {total_files} phần trong thư mục {output_folder}.")



NOI NHIEU VAN BAN LUAT

In [16]:
import os
import pythoncom
import time
from win32com.client import Dispatch

def merge_doc_files(input_folder, output_file):
    doc_files = [f for f in os.listdir(input_folder) if f.endswith('.doc')]
    doc_files.sort() 

    pythoncom.CoInitialize()
    word = Dispatch("Word.Application")
    word.Visible = False 

    merged_doc = word.Documents.Add()

    for doc_file in doc_files:
        file_path = os.path.join(input_folder, doc_file)
        temp_doc = word.Documents.Open(file_path)
        temp_doc.Content.Copy()
        merged_doc.Range(merged_doc.Content.End - 1).Paste()
        merged_doc.Range(merged_doc.Content.End - 1).InsertBreak(2)  # Thêm ngắt trang giữa các tài liệu
        temp_doc.Close()
        
        time.sleep(0.5)
    
    merged_doc.SaveAs(output_file)
    merged_doc.Close()
    word.Quit()

    print(f"Đã hợp nhất {len(doc_files)} file vào {output_file}")

In [12]:
def clear_licence_in_txt(input_dir):
    lines_to_remove = 2  

    for file_name in os.listdir(input_dir):
        if file_name.lower().endswith('.txt'):
            file_path = os.path.join(input_dir, file_name)

            with open(file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
            
            lines = lines[lines_to_remove:]

            with open(file_path, 'w', encoding='utf-8') as file:
                file.writelines(lines)
        

TÁCH THÀNH TẬP TRAIN VÀ TEST

In [4]:
import random
def copy_file(src_path, dest_path):
    with open(src_path, 'rb') as src_file:
        with open(dest_path, 'wb') as dest_file:
            dest_file.write(src_file.read())

def split_train_test(input_dir, output_dir,title ,train_ratio=0.7):
    os.makedirs(output_dir, exist_ok=True)

    # Tạo các thư mục train và test
    train_dir = os.path.join(output_dir, title +'train')
    test_dir = os.path.join(output_dir, title +'test')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    txt_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.txt')]
    
    random.shuffle(txt_files)
    
    split_index = int(len(txt_files) * train_ratio)
    
    train_files = txt_files[:split_index]
    test_files = txt_files[split_index:]
    
    def copy_files(files, target_dir):
        for file_name in files:
            src_path = os.path.join(input_dir, file_name)
            dest_path = os.path.join(target_dir, file_name)
            try:
                copy_file(src_path, dest_path)
                print(f"Đã sao chép {src_path} đến {dest_path}")
            except Exception as e:
                print(f"Error copying {src_path}: {e}")

    copy_files(train_files, train_dir)
    copy_files(test_files, test_dir)

In [14]:
def download_files_from_pages1(dir, base_url_template, start_page=1, end_page=5):
    if not os.path.exists(dir):
        os.makedirs(dir)
    
    for page in range(start_page, end_page + 1):
        base_url = base_url_template.format(page=page)
        print(f"Đang xử lý trang: {base_url}")

        response = requests.get(base_url)
        response.raise_for_status()  # Kiểm tra lỗi HTTP
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        attachment_links = soup.find_all('a', href=True)
        
        file_links = []
        for link in attachment_links:
            href = link['href']
            if 'javascript:downloadfile' in href:
                match = re.search(r"downloadfile\('([^']+)',\s*'([^']+)'\)", href)
                if match:
                    file_name = match.group(1)
                    file_url = f"https://vbpl.vn{match.group(2)}"
                    if not file_name.lower().endswith('.zip'):
                        file_links.append((file_url, file_name))
        
        print(f"Tìm thấy các liên kết trên trang: {file_links}")

        for file_url, file_name in file_links:
            save_path = os.path.join(dir, file_name)
            
            print(f"Tải xuống: {file_url}")
            
            if not os.path.exists(save_path):
                try:
                    file_response = requests.get(file_url)
                    file_response.raise_for_status()  # Kiểm tra lỗi HTTP
                    with open(save_path, 'wb') as file:
                        file.write(file_response.content)
                    print(f"Đã tải xuống {file_url} và lưu tại {save_path}")
                except requests.RequestException as e:
                    print(f"Lỗi khi tải {file_url}: {e}")

DAN SU

In [27]:
input_file = 'D:\\Private\\DATA\\luat\\dan_su\\dan_su_text\\bo_luat_dan_su.txt'
output_folder = 'D:\\Private\\DATA\\law_data\\dan_su'
split_txt_by_rules(input_file,output_folder)

# input_dir = 'D:\Private\DATA\law_data\dan_su'
# output_dir = 'D:\Private\DATA\law_data_test'
# split_train_test(input_dir, output_dir, train_ratio=0.7)

Đã tách file D:\Private\DATA\luat\dan_su\dan_su_text\bo_luat_dan_su.txt thành 437 phần trong thư mục D:\Private\DATA\law_data\dan_su.


In [28]:

input_dir = 'D:\Private\DATA\law_data\dan_su'
output_dir = 'D:\Private\DATA\DATA_LAW'
train_ratio = 0.7
split_train_test(input_dir, output_dir,'dan_su', train_ratio)

Đã sao chép D:\Private\DATA\law_data\dan_su\Dieu_147.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Dieu_147.txt
Đã sao chép D:\Private\DATA\law_data\dan_su\Dieu_286.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Dieu_286.txt
Đã sao chép D:\Private\DATA\law_data\dan_su\Dieu_48.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Dieu_48.txt
Đã sao chép D:\Private\DATA\law_data\dan_su\Dieu_315.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Dieu_315.txt
Đã sao chép D:\Private\DATA\law_data\dan_su\Dieu_70.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Dieu_70.txt
Đã sao chép D:\Private\DATA\law_data\dan_su\Dieu_126.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Dieu_126.txt
Đã sao chép D:\Private\DATA\law_data\dan_su\Dieu_154.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Dieu_154.txt
Đã sao chép D:\Private\DATA\law_data\dan_su\Dieu_39.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Dieu_39.txt
Đã sao chép D:\Private\DATA\law_data\dan_su\Dieu_231.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Dieu_231.txt
Đã sao 

HINH SU

In [11]:
input_file = 'D:\\Private\\DATA\\luat\\hinh_su\\luat_hinh_su.doc'
output_folder = 'D:\\Private\\DATA\\law_data\\hinh_su'
split_doc_by_rules(input_file,output_folder)



Đã tách file D:\Private\DATA\luat\hinh_su\luat_hinh_su.doc thành 779 phần trong thư mục D:\Private\DATA\law_data\hinh_su.


In [19]:

input_dir = 'D:\Private\DATA\law_data\hinh_su'
output_dir = 'D:\Private\DATA\DATA_LAW'
train_ratio = 0.7
split_train_test(input_dir, output_dir,'hinh_su ', train_ratio)


Đã sao chép D:\Private\DATA\law_data\hinh_su\Dieu_754.txt đến D:\Private\DATA\DATA_LAW\hinh_su train\Dieu_754.txt
Đã sao chép D:\Private\DATA\law_data\hinh_su\Dieu_39.txt đến D:\Private\DATA\DATA_LAW\hinh_su train\Dieu_39.txt
Đã sao chép D:\Private\DATA\law_data\hinh_su\Dieu_416.txt đến D:\Private\DATA\DATA_LAW\hinh_su train\Dieu_416.txt
Đã sao chép D:\Private\DATA\law_data\hinh_su\Dieu_277.txt đến D:\Private\DATA\DATA_LAW\hinh_su train\Dieu_277.txt
Đã sao chép D:\Private\DATA\law_data\hinh_su\Dieu_732.txt đến D:\Private\DATA\DATA_LAW\hinh_su train\Dieu_732.txt
Đã sao chép D:\Private\DATA\law_data\hinh_su\Dieu_556.txt đến D:\Private\DATA\DATA_LAW\hinh_su train\Dieu_556.txt
Đã sao chép D:\Private\DATA\law_data\hinh_su\Dieu_93.txt đến D:\Private\DATA\DATA_LAW\hinh_su train\Dieu_93.txt
Đã sao chép D:\Private\DATA\law_data\hinh_su\Dieu_66.txt đến D:\Private\DATA\DATA_LAW\hinh_su train\Dieu_66.txt
Đã sao chép D:\Private\DATA\law_data\hinh_su\Dieu_510.txt đến D:\Private\DATA\DATA_LAW\hinh_su

LAO DONG

In [10]:
input_file = 'D:\\Private\\DATA\\luat\\lao_dong\\luat_lao_dong.docx'
output_folder = 'D:\\Private\\DATA\\law_data\\lao_dong'
split_doc_by_rules(input_file,output_folder)

Đã tách file D:\Private\DATA\luat\lao_dong\luat_lao_dong.docx thành 221 phần trong thư mục D:\Private\DATA\law_data\lao_dong.


In [20]:

input_dir = 'D:\Private\DATA\law_data\lao_dong'
output_dir = 'D:\Private\DATA\DATA_LAW'
train_ratio = 0.7
split_train_test(input_dir, output_dir,'lao_dong', train_ratio)


Đã sao chép D:\Private\DATA\law_data\lao_dong\Dieu_142.txt đến D:\Private\DATA\DATA_LAW\lao_dongtrain\Dieu_142.txt
Đã sao chép D:\Private\DATA\law_data\lao_dong\Dieu_80.txt đến D:\Private\DATA\DATA_LAW\lao_dongtrain\Dieu_80.txt
Đã sao chép D:\Private\DATA\law_data\lao_dong\Dieu_73.txt đến D:\Private\DATA\DATA_LAW\lao_dongtrain\Dieu_73.txt
Đã sao chép D:\Private\DATA\law_data\lao_dong\Dieu_16.txt đến D:\Private\DATA\DATA_LAW\lao_dongtrain\Dieu_16.txt
Đã sao chép D:\Private\DATA\law_data\lao_dong\Dieu_27.txt đến D:\Private\DATA\DATA_LAW\lao_dongtrain\Dieu_27.txt
Đã sao chép D:\Private\DATA\law_data\lao_dong\Dieu_145.txt đến D:\Private\DATA\DATA_LAW\lao_dongtrain\Dieu_145.txt
Đã sao chép D:\Private\DATA\law_data\lao_dong\Dieu_4.txt đến D:\Private\DATA\DATA_LAW\lao_dongtrain\Dieu_4.txt
Đã sao chép D:\Private\DATA\law_data\lao_dong\Dieu_68.txt đến D:\Private\DATA\DATA_LAW\lao_dongtrain\Dieu_68.txt
Đã sao chép D:\Private\DATA\law_data\lao_dong\Dieu_63.txt đến D:\Private\DATA\DATA_LAW\lao_don

DAT_DAI

In [7]:

# Ví dụ cách sử dụng
input_file = "D:\\Private\\DATA\\data_crawl\\dat_dai\\dat_dai.doc"
output_folder = "D:\\Private\\DATA\\data_test\\dat_dai"

split_doc_by_rules(input_file, output_folder)

Đã tách file D:\Private\DATA\luat\dat_dai\dat_dai.doc thành 258 phần trong thư mục D:\Private\DATA\law_data\dat_dai.


In [21]:
input_dir = 'D:\Private\DATA\data_test\dat_dai'
output_dir = 'D:\Private\DATA\DATA_LAW'
train_ratio = 0.7
split_train_test(input_dir, output_dir,'dat_dai', train_ratio)

Đã sao chép D:\Private\DATA\law_data\dat_dai\Dieu_225.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Dieu_225.txt
Đã sao chép D:\Private\DATA\law_data\dat_dai\Dieu_17.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Dieu_17.txt
Đã sao chép D:\Private\DATA\law_data\dat_dai\Dieu_227.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Dieu_227.txt
Đã sao chép D:\Private\DATA\law_data\dat_dai\Dieu_75.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Dieu_75.txt
Đã sao chép D:\Private\DATA\law_data\dat_dai\Dieu_237.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Dieu_237.txt
Đã sao chép D:\Private\DATA\law_data\dat_dai\Dieu_87.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Dieu_87.txt
Đã sao chép D:\Private\DATA\law_data\dat_dai\Dieu_186.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Dieu_186.txt
Đã sao chép D:\Private\DATA\law_data\dat_dai\Dieu_40.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Dieu_40.txt
Đã sao chép D:\Private\DATA\law_data\dat_dai\Dieu_7.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Dieu_7

Kinh Te


In [14]:
merge_doc_files('D:\\Private\\DATA\\luat\\kinh_te','D:\\Private\\DATA\\luat\\kinh_te\\kinh_te.doc')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\Private\\DATA\\luat\\kinh_te'

In [6]:
# Ví dụ cách sử dụng
input_file = "D:\\Private\\DATA\\luat\\kinh_te\\kinh_te.doc"
output_folder = "D:\\Private\\DATA\\law_data\\kinh_te"

split_doc_by_rules(input_file, output_folder)

Đã tách file D:\Private\DATA\luat\kinh_te\kinh_te.doc thành 678 phần trong thư mục D:\Private\DATA\law_data\kinh_te.


In [22]:
input_dir = 'D:\Private\DATA\law_data\kinh_te'
output_dir = 'D:\Private\DATA\DATA_LAW'
train_ratio = 0.7
split_train_test(input_dir, output_dir,'kinh_te', train_ratio)

Đã sao chép D:\Private\DATA\law_data\kinh_te\Dieu_478.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrain\Dieu_478.txt
Đã sao chép D:\Private\DATA\law_data\kinh_te\Dieu_166.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrain\Dieu_166.txt
Đã sao chép D:\Private\DATA\law_data\kinh_te\Dieu_138.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrain\Dieu_138.txt
Đã sao chép D:\Private\DATA\law_data\kinh_te\Dieu_430.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrain\Dieu_430.txt
Đã sao chép D:\Private\DATA\law_data\kinh_te\Dieu_173.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrain\Dieu_173.txt
Đã sao chép D:\Private\DATA\law_data\kinh_te\Dieu_237.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrain\Dieu_237.txt
Đã sao chép D:\Private\DATA\law_data\kinh_te\Dieu_533.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrain\Dieu_533.txt
Đã sao chép D:\Private\DATA\law_data\kinh_te\Dieu_15.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrain\Dieu_15.txt
Đã sao chép D:\Private\DATA\law_data\kinh_te\Dieu_557.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrai

In [26]:
def doc_to_txt(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    # Lấy tên thư mục gốc để làm tên tệp
    folder_name = os.path.basename(os.path.normpath(input_dir))

    doc_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.doc')]
    
    # Đánh số thứ tự bắt đầu từ 1
    for i, file_name in enumerate(doc_files, start=1):
        doc_path = os.path.join(input_dir, file_name)
        txt_filename = f'{folder_name}_{i}.txt'
        txt_path = os.path.join(output_dir, txt_filename)
        try:
            doc = aw.Document(doc_path)
            doc.save(txt_path)
            print(f"Đã chuyển đổi {doc_path} và lưu tại {txt_path}")
        except Exception as e:
            print(f"Error processing {doc_path}: {e}")


In [27]:
doc_to_txt('D:\Private\DATA\luat\kinh_te','D:\Private\DATA\luat\kinh_te')

Đã chuyển đổi D:\Private\DATA\luat\kinh_te\chung_khoan.doc và lưu tại D:\Private\DATA\luat\kinh_te\kinh_te_1.txt
Đã chuyển đổi D:\Private\DATA\luat\kinh_te\kinh_te.doc và lưu tại D:\Private\DATA\luat\kinh_te\kinh_te_2.txt
Đã chuyển đổi D:\Private\DATA\luat\kinh_te\Kinh_te_chung_khoan.doc và lưu tại D:\Private\DATA\luat\kinh_te\kinh_te_3.txt
Đã chuyển đổi D:\Private\DATA\luat\kinh_te\Kinh_te_doanh_nghiep.doc và lưu tại D:\Private\DATA\luat\kinh_te\kinh_te_4.txt
Đã chuyển đổi D:\Private\DATA\luat\kinh_te\KInh_te_thuong_mai.doc và lưu tại D:\Private\DATA\luat\kinh_te\kinh_te_5.txt


DAU THAU

In [3]:
# Ví dụ cách sử dụng
input_file = "D:\Private\DATA\luat\dau_thau\dau_thau.doc"
output_folder = "D:\Private\DATA\law_data\dau_thau"

split_doc_by_rules(input_file, output_folder)


Đã tách file D:\Private\DATA\luat\dau_thau\dau_thau.doc thành 96 phần trong thư mục D:\Private\DATA\law_data\dau_thau.


In [23]:

input_dir = 'D:\Private\DATA\law_data\dau_thau'
output_dir = 'D:\Private\DATA\DATA_LAW'
train_ratio = 0.7
split_train_test(input_dir, output_dir,'dau_thau', train_ratio)

Đã sao chép D:\Private\DATA\law_data\dau_thau\Dieu_58.txt đến D:\Private\DATA\DATA_LAW\dau_thautrain\Dieu_58.txt
Đã sao chép D:\Private\DATA\law_data\dau_thau\Dieu_11.txt đến D:\Private\DATA\DATA_LAW\dau_thautrain\Dieu_11.txt
Đã sao chép D:\Private\DATA\law_data\dau_thau\Dieu_39.txt đến D:\Private\DATA\DATA_LAW\dau_thautrain\Dieu_39.txt
Đã sao chép D:\Private\DATA\law_data\dau_thau\Dieu_89.txt đến D:\Private\DATA\DATA_LAW\dau_thautrain\Dieu_89.txt
Đã sao chép D:\Private\DATA\law_data\dau_thau\Dieu_82.txt đến D:\Private\DATA\DATA_LAW\dau_thautrain\Dieu_82.txt
Đã sao chép D:\Private\DATA\law_data\dau_thau\Dieu_13.txt đến D:\Private\DATA\DATA_LAW\dau_thautrain\Dieu_13.txt
Đã sao chép D:\Private\DATA\law_data\dau_thau\Dieu_37.txt đến D:\Private\DATA\DATA_LAW\dau_thautrain\Dieu_37.txt
Đã sao chép D:\Private\DATA\law_data\dau_thau\Dieu_71.txt đến D:\Private\DATA\DATA_LAW\dau_thautrain\Dieu_71.txt
Đã sao chép D:\Private\DATA\law_data\dau_thau\Dieu_7.txt đến D:\Private\DATA\DATA_LAW\dau_thautr

HANG HAI

In [17]:
merge_doc_files('D:\Private\DATA\data_to_test\hang_hai','D:\Private\DATA\data_to_test\hang_hai')

com_error: (-2147417848, 'The object invoked has disconnected from its clients.', None, None)

In [9]:
input_dir = 'D:\Private\DATA\data_to_test\hang_hai'
output_dir = 'D:\Private\DATA\DATA_LAW'
train_ratio = 0.7
rename_file(input_dir)
split_train_test(input_dir, output_dir,'hang_hai', train_ratio)